# Boilerplate

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio
import time

from nwv2_python_wrapper import *
import nwv2_python_wrapper

# Run

List all Python wrappers

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['BroadcastChatMessageW',
 'EndpointW',
 'FilterCmdW',
 'FilterInterface',
 'FilterModeW',
 'FilterNoticeW',
 'FilterRspW',
 'GameOptionsW',
 'GameOutcomeW',
 'GameUpdateW',
 'GenPartInfoW',
 'GenStateDiffPartW',
 'GenStateDiffW',
 'NetRegionW',
 'PacketSettingsW',
 'PacketW',
 'PlayerInfoW',
 'ProcessUniqueIdW',
 'RequestActionW',
 'ResponseCodeW',
 'RoomListW',
 'TransportCmdW',
 'TransportInterface',
 'TransportNoticeW',
 'TransportRspW',
 'UniUpdateW',
 'new_transport_interface']

## Transport layer setup

Create the Transport layer and run it in the background.

In [3]:
tiface_inner = await new_transport_interface("0.0.0.0", 12345)
tiface_inner.run()

<Future pending cb=[<builtins.PyDoneCallback object at 0x7fbc8e1ef330>()]>

## Wrap the PYO3 Transport Interface in a Python class with the same methods

Note that there is no `run()`. Hence, the passed in `TransportInterface` instance must have already had its `run()` method called.

In [4]:
class LoggingTransportInterface:
    def __init__(self, inner):
        self.inner = inner
        
    async def command_response(self, transport_cmd):
        print(f"LTI: received transport_cmd {transport_cmd}")
        transport_rsp = await self.inner.command_response(transport_cmd)
        print(f"LTI: got transport_rsp back from Transport layer: {transport_rsp}")
        return transport_rsp
    
    def get_notifications(self):
        transport_notif_list = self.inner.get_notifications()
        print(f"LTI: got transport notifications: {transport_notif_list}")
        return transport_notif_list
        
tiface = LoggingTransportInterface(tiface_inner)
tiface

# Filter layer setup

In [5]:
fiface = FilterInterface(tiface, FilterModeW("client"))

Find methods to run

In [6]:
[m for m in dir(fiface) if not m.startswith('__')]

['command_response', 'get_notifications', 'notif_poll_ms', 'run']

Run Filter!

In [7]:
run_fut = fiface.run()

# After waiting a bit, the above future should not have completed
time.sleep(0.1)
assert not run_fut.done()

LTI: got transport notifications: []
LTI: got transport notifications: []
LTI: got transport notifications: []
LTI: got transport notifications: []


Get the number of milliseconds between calling `get_notifications` on the contained `TransportInterface`. This can also be assigned to at any time. Lower values can cause performance issues, while higher values add latency for anything involving filter notifications.

In [8]:
fiface.notif_poll_ms

30

In [9]:
fiface.notif_poll_ms = 100

## Filter layer operation

### As a client, connect to server

In [10]:
import random
id = random.randint(1,10000)
connect_action = RequestActionW("connect", name="FilterNotebookClient{}".format(id), client_version="0.0.666")
connect_action

Connect { name: "FilterNotebookClient9991", client_version: "0.0.666" }

In [11]:
chococat_endpoint = EndpointW("157.230.134.224:2016")
filter_cmd = FilterCmdW("sendrequestaction", endpoint=chococat_endpoint, action=connect_action)
filter_cmd

LTI: got transport notifications: []


SendRequestAction { endpoint: Endpoint(157.230.134.224:2016), action: Connect { name: "FilterNotebookClient9991", client_version: "0.0.666" } }

In [12]:
await fiface.command_response(filter_cmd)

LTI: received transport_cmd SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 0 }, retry_interval: 5ms }], packets: [Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient9991", client_version: "0.0.666" } }] }
LTI: got transport_rsp back from Transport layer: None


thread 'tokio-runtime-worker' panicked at 'command_response must return TransportRspW: PyErr { type: <class 'TypeError'>, value: TypeError("'NoneType' object cannot be converted to 'TransportRspW'"), traceback: None }', nwv2-python-wrapper/src/filter/mod.rs:267:63
stack backtrace:
   0: rust_begin_unwind
             at /rustc/897e37553bba8b42751c67658967889d11ecd120/library/std/src/panicking.rs:584:5
   1: core::panicking::panic_fmt
             at /rustc/897e37553bba8b42751c67658967889d11ecd120/library/core/src/panicking.rs:142:14
   2: core::result::unwrap_failed
             at /rustc/897e37553bba8b42751c67658967889d11ecd120/library/core/src/result.rs:1785:5
   3: core::result::Result<T,E>::expect
             at /rustc/897e37553bba8b42751c67658967889d11ecd120/library/core/src/result.rs:1064:23
   4: nwv2_python_wrapper::filter::handle_transport_cmd_resp::{{closure}}
             at ./src/filter/mod.rs:267:13
   5: <core::future::from_generator::GenFuture<T> as core::future::future

In [13]:
fiface.get_notifications()

Exception: filter notify channel was disconnected

In [ ]:
new_response_code = _[0]
new_response_code

In [ ]:
cookie = new_response_code.response_code.cookie
cookie

### Convenience function

In [ ]:
async def request_action_to_filter(req_action, endpoint=chococat_endpoint):
    filter_cmd = FilterCmdW("sendrequestaction", endpoint=endpoint, action=req_action)
    return await fiface.command_response(filter_cmd)

### Join a room

In [ ]:
await request_action_to_filter(RequestActionW("joinroom", room_name="general"))

Due to a bug in the Netwayste v1 server, there is a `BadRequest { error_msg: "cannot join game because in-game" }` filter notice following the `JoinedRoom` filter notice, below. Apparently, it's caused by a packet retry of the `JoinRoom` request action by the Transport layer, which is erroneously being treated as a second request action by the NWv1 server. It's erroneous because the request sequence is the same, so it should have been dropped by the server.

In [ ]:
fiface.get_notifications()